In [1]:
import os
import requests
import datetime
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
options = Options()
browser = Chrome(ChromeDriverManager().install(), options=options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 110.0.5481
[WDM] - Get LATEST driver version for 110.0.5481


[WDM] - Driver [/Users/natalie/.wdm/drivers/chromedriver/mac64/110.0.5481.77/chromedriver] found in cache


## Collect All Reviews for a Reviewer

In [ ]:
userhandle = "DanielFoodDiary" ### TO CHANGE HERE
URL = "https://www.burpple.com/@" + userhandle + "/timeline"

In [ ]:
browser.get(URL)

In [ ]:
load_more = True
while load_more:
    try:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        more_reviews = browser.find_element_by_id("masonryViewMore-btn")
        time.sleep(1)
        more_reviews.click()
        time.sleep(1)
    except:
        load_more = False

In [ ]:
# find all reviews by a given revier
page_html = BeautifulSoup(browser.page_source, "html.parser")
all_reviews = page_html.findAll(class_="food card feed-item")

In [ ]:
reviews_by_reviewer = []
for review_listing in all_reviews:
    # name and address
    header = review_listing.find(class_="food-place card-item card-item--header")
    try: 
        name = header.find(class_="card-item-set--link-title").text
        name = name.replace('\n', '')

        address = review_listing.find(class_="card-item-set--link-subtitle").text
        address = address.replace('\n', '')
    except:
        name = None
        address = None

    # review
    review = review_listing.find(class_="food-description").text

    # date
    user_card = review_listing.find(class_="food-user card-item")
    try:
        date = user_card.find(class_="card-item-set--link-subtitle").text
        date = date.replace('\n', '')
    except:
        date = None

    # restaurant listing link
    link = review_listing.find_all("a", href=True)[0]['href']
    link = 'https://www.burpple.com' + link

    reviews_by_reviewer.append([name, address, review, date, link])

In [ ]:
reviews_by_reviewer_df = pd.DataFrame(reviews_by_reviewer, columns=['name', 'address', 'review', 'date', 'link'])

In [ ]:
reviews_by_reviewer_df.head(3)

In [ ]:
reviews_by_reviewer_df.to_csv("reviewer-data/" + userhandle + "_reviews.csv")